# Ryhmä-190

## Python-paketit

In [37]:
# Jupyter-notebookissa voi asentaa paketit samaan tapaan kuin komentoriviltä.
# Tähän käytetään "magic commandeja", jotka alkavat %-merkillä.
# Näiden asentamisessa voi mennä muutama minuutti, mutta sen jälkeen niitä ei tarvitse asentaa uudestaan. 
#%pip install seaborn
#%pip install mlflow azureml azureml-core azureml-mlflow azure-identity

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
from sklearn.model_selection import KFold, cross_val_score
#from azureml.core import Workspace
#from azure.identity import InteractiveBrowserCredential
#import mlflow

## MLFlow-seuranta (ei tällä hetkellä käytössä)

Tarkistetaan ajoympäristö (Azure vai oma kone) ja autentikoidutaan sen mukaan.

In [39]:
def is_running_in_azure():
    return 'AZUREML_RUN_ID' in os.environ

def get_workspace():
    if is_running_in_azure():
        return Workspace.from_config()
    else:
        return Workspace(subscription_id='1c0e26b6-0fcb-4b6d-911c-2a0836275ea4',
                         resource_group='rg-AML',
                         workspace_name='aml-data_science_masters')
#ws = get_workspace()

In [40]:
#mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
#experiment_name = 'group-190-tracking'
#mlflow.set_experiment(experiment_name)
#mlflow.autolog()

In [73]:
def evaluate_model(model, X_train, y_train, random_state=42):
    ### Train loss
    y_train_pred = model.predict(X_train)
    train_loss = mean_squared_error(y_train, y_train_pred)

    ### 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    mse_scorer = make_scorer(mean_squared_error)
    cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring=mse_scorer)
    cv_loss_mean = cv_scores.mean()

    r2_train = r2_score(y_train, y_train_pred)
    r2_cv = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2').mean()

    print('Train loss:', train_loss)
    print('CV loss mean:', cv_loss_mean)
    print('Train R^2:', r2_train)
    print('CV R^2:', r2_cv)

def transform_gbr(df):
    X_train_gbr = df.copy()
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)

    # Add meaningful features
    X_train_gbr['AtomFraction'] = (X_train_gbr['NumOfC'] + X_train_gbr['NumOfO'] + X_train_gbr['NumOfN']) / X_train_gbr['NumOfAtoms']
    X_train_gbr['Polarity'] = X_train_gbr['NumHBondDonors'] / X_train_gbr['MW']
    X_train_gbr['HBondDensity'] = df['NumHBondDonors'] / X_train_gbr['NumOfAtoms']
    X_train_gbr['GroupDensity_CarboxylicAcid'] = X_train_gbr['carboxylic acid'] / X_train_gbr['MW']
    X_train_gbr['Unsaturation'] = X_train_gbr['C=C (non-aromatic)'] + X_train_gbr['C=C-C=O in non-aromatic ring']
    X_train_gbr['ConfigurationalComplexity'] = X_train_gbr['NumOfConf'] / X_train_gbr['MW']
    # X_train_gbr['NumOfConf'] = np.log(X_train_gbr['NumOfConf'])
    # X_train_gbr['NumOfConfUsed'] = (X_train_gbr['NumOfConfUsed'] == 40).astype(int)
    drop_cols = [
        'NumOfC',
        'NumOfO',
        'NumOfN',
        'NumOfAtoms',
        'NumHBondDonors',
        'carboxylic',
        'MW',
        'C=C-C=O in non-aromatic ring',
        'C=C (non-aromatic)',
        'NumOfConf',
        'NumOfConfUsed',
    ]
    X_train_gbr.drop(columns=drop_cols, axis=1)

    return X_train_gbr

## Tietoaineistojen lataaminen

In [67]:
df_train = pd.read_csv('../data/train.csv', encoding='utf-8', header=0)
df_test = pd.read_csv('../data/test.csv', encoding='utf-8', header=0)

## Esiprosessointi

In [69]:
df_train['parentspecies'] = df_train['parentspecies'].astype('category')
df_train['parentspecies'] = df_train['parentspecies'].cat.codes
df_test['parentspecies'] = df_test['parentspecies'].astype('category')
df_test['parentspecies'] = df_test['parentspecies'].cat.codes

## Mallien kouluttaminen

In [59]:
X_train, y_train = df_train.drop(columns=['log_pSat_Pa', 'ID']), df_train['log_pSat_Pa']
X_test = df_test.drop(columns=['ID'])
X_train_gbr = transform_gbr(X_train)
X_test_gbr = transform_gbr(X_test)
X_test_gbr.head()

,MW,NumOfAtoms,NumOfC,NumOfO,NumOfN,NumHBondDonors,NumOfConf,NumOfConfUsed,parentspecies,C=C (non-aromatic),...,hydroperoxide,carbonylperoxyacid,nitroester,log_pSat_Pa,AtomFraction,Polarity,HBondDensity,GroupDensity_CarboxylicAcid,Unsaturation,ConfigurationalComplexity
0,327.966253,26,6,14,2,2,5.579730,39.0,5,0,...,0,1,0,-5.747870,0.846154,0.006098,0.076923,0.003049,0,0.808010
1,361.971732,30,6,16,2,2,5.683580,40.0,5,0,...,2,0,0,-3.922929,0.800000,0.005525,0.066667,0.000000,0,0.812218
2,237.012081,23,6,9,1,2,4.234107,12.0,5,0,...,0,1,0,-3.958431,0.695652,0.008438,0.086957,0.000000,0,0.291124
3,342.054674,37,9,12,2,2,5.605802,16.0,0,0,...,1,1,0,-6.245400,0.621622,0.005847,0.054054,0.000000,0,0.795195
4,311.971338,25,6,13,2,1,4.369448,34.0,5,0,...,0,0,0,-1.041082,0.840000,0.003205,0.040000,0.000000,0,0.253228


Erilaisia malleja alla. Kaikki mallit käyttävät samaa nimeä `model`, eli vain viimeisenä koulutettu menee testeihin ja tallentuu.

### 1. Dummy

In [45]:
model = DummyRegressor(strategy='mean')
model.fit(X_train, y_train)

df_test['log_pSat_Pa'] = model.predict(X_test)

### 2. Yksinkertainen regressio

In [ ]:
model = LinearRegression()
model.fit(X_train_gbr, y_train)

evaluate_model(model, X_train_gbr, y_train)

Train loss: 2.800073513771292
CV loss mean: 2.807719632298041
Train R^2: 0.7123772224846023
CV R^2: 0.7114160415718718


### 3. Random Forest

In [61]:
model = RandomForestRegressor(random_state=190)
model.fit(X_train_gbr, y_train)

evaluate_model(model, X_train_gbr, y_train)

Train loss: 0.3863467117691719
CV loss mean: 2.7737210268490045
Train R^2: 0.9603145725365886
CV R^2: 0.714920252540877


### 4. Gradient Boosting Regressor

In [ ]:
gbr_params = {
    "n_estimators": 100,
    "max_depth": 6,
    "min_samples_split": 10,
    "learning_rate": 0.1,
    "loss": "squared_error",
    "random_state": 42,
    # "subsample": 0.6,
}
    
model = GradientBoostingRegressor(**gbr_params)
model.fit(X_train_gbr, y_train)

evaluate_model(model, X_train_gbr, y_train)



Train loss: 1.4859814893458567
CV loss mean: 2.5368206932965776
Train R^2: 0.8473603920753945
CV R^2: 0.7392494907504896


In [ ]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR

model = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2, kernel='rbf')),
model.fit(X_train_gbr, y_train)

evaluate_model(model, X_train_gbr, y_train)

AttributeError: 'tuple' object has no attribute 'fit'

## Mallien arviointi

In [50]:
### Train loss
y_train_pred = model.predict(X_train)
train_loss = mean_squared_error(y_train, y_train_pred)

### 5-fold cross-validation
results = []
kf = KFold(n_splits=5, shuffle=True, random_state=190)
mse_scorer = make_scorer(mean_squared_error)
cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring=mse_scorer)
cv_loss_mean = cv_scores.mean()

r2_train = r2_score(y_train, y_train_pred)
r2_cv = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2').mean()

print('Train loss:', train_loss)
print('CV loss mean:', cv_loss_mean)
print('Train R^2:', r2_train)
print('CV R^2:', r2_cv)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- AtomFraction
- ConfigurationalComplexity
- GroupDensity_CarboxylicAcid
- HBondDensity
- Polarity
- ...


## Ennustuksen tallentaminen

In [75]:
df_test['log_pSat_Pa'] = model.predict(X_test_gbr.drop(columns=['log_pSat_Pa'], axis=1))
df_test[['ID', 'log_pSat_Pa']].to_csv('../submission/submission.csv', index=False)